# Lab 7 Word Sequences

Lab session by:
* Daniel Hess
* Pandelis Laurens Symeonidis

## Imports

In [18]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import jaccard_distance
from scipy.stats import pearsonr
import spacy

### Might need to run:
```
!pip install spacy
!python3 -m spacy download en_core_web_sm

In [19]:
!pip install spacy
!python3 -m spacy download en_core_web_sm


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Daniel Hess\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 640.0 kB/s eta 0:00:20
     ---------------------------------------- 0.1/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.7/12.8 MB 5.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.8 MB 8.3 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/12.8 MB 9.7 MB/s eta 0:00:02
     ------- -------------------------------- 2.5/12.8 MB 9.2 MB/s eta 0:00:02
     --------- ------------------------------ 3.2/12.8 MB 10.1 MB/s eta 0:00:01
     ---------- ----------------------------- 3.5/12.8 MB 9.7 MB/s eta 0:00:01
     ------------ --------------------------- 4.1/12.8 MB 10.1 MB/s eta 0:00:01
     -------------- ------------------------- 4.6/12.8 MB 10.2 MB/s eta 0:00:01
     ---------------- ----------------------- 5.2/12.8 MB 10.7 MB/s eta 0:00:01
     ------------------ --------------------- 5.9/12.8


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Daniel Hess\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Load Data

In [20]:
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('average_perceptron_tagger_eng', quiet=True)
nltk.download('maxent_ne_chunker', quiet=True)
nltk.download('words', quiet=True)

nlp = spacy.load("en_core_web_sm")

# Load dataframes
dt = pd.read_csv('./STS.input.SMTeuroparl.txt', sep='\t', header=None)
gold_standard_dt = pd.read_csv('./STS.gs.SMTeuroparl.txt', sep='\t', header=None)

[nltk_data] Error loading average_perceptron_tagger_eng: Package
[nltk_data]     'average_perceptron_tagger_eng' not found in index


In [ ]:
def NE_nltk(sentence):
    tokens =  word_tokenize(sentence)

    penn_tagged = nltk.pos_tag(tokens)
    tree = nltk.ne_chunk(penn_tagged)

    res = [t.lower() for t in tokens]

    for node in tree:
        if isinstance(node, nltk.Tree):
            words = [w for w, t in node.leaves()]
            res.append(" ".join(words).lower())

    return res

def NE_spaCy(sentence):
    res = [t.lower() for t in word_tokenize(sentence)]

    doc = nlp(sentence)

    for ent in doc.ents:
        res.append(ent.text.lower())

    return res


def getNEs(dt, method='nltk'):

    word_and_NEs1 = []
    word_and_NEs2 = []

    if method == 'nltk':

        word_and_NEs1 = dt[0].apply(NE_nltk)
        word_and_NEs2 = dt[1].apply(NE_nltk)

    elif method == 'spaCy':
        word_and_NEs1 = dt[0].apply(NE_spaCy)
        word_and_NEs2 = dt[1].apply(NE_spaCy)

    elif method == 'TextServer':
        raise NotImplementedError

    return word_and_NEs1, word_and_NEs2


In [ ]:
def prep_sentences(dt, lower=False, regex=False):
    sents1 = dt[0].apply(word_tokenize)
    sents2 = dt[1].apply(word_tokenize)

    if lower:
        sents1 = [t.lower() for t in sents1]
        sents2 = [t.lower() for t in sents1]
    
    return sents1, sents2
    
word_and_NE_nltk1, word_and_NE_nltk2 = getNEs(dt.copy(), method='nltk')
word_and_NE_spaCy1, word_and_NE_spaCy2 = getNEs(dt.copy(), method='spaCy')
w1, s2 = prep_sentences(dt.copy(), lower=True)

jaccard_distances_NE_nltk = [jaccard_distance(set(s1), set(s2)) for s1, s2 in zip(word_and_NE_nltk1, word_and_NE_nltk2)]
similarities_NE_nltk = [1 - d for d in jaccard_distances_NE_nltk]

jaccard_distances_NE_spaCy = [jaccard_distance(set(s1), set(s2)) for s1, s2 in zip(word_and_NE_spaCy1, word_and_NE_spaCy2)]
similarities_NE_spaCy = [1 - d for d in jaccard_distances_NE_spaCy]

jaccard_distances = [jaccard_distance(set(s1), set(s2)) for s1, s2 in zip(w1, s2)]
similarities = [1 - d for d in jaccard_distances]

corr1 = pearsonr(similarities_NE_nltk, gold_standard_dt[0])[0]
corr2 = pearsonr(similarities_NE_spaCy, gold_standard_dt[0])[0]
corr3 = pearsonr(similarities, gold_standard_dt[0])[0]

print(corr1)
print(corr2)
print(corr3)

TypeError: 'method' object is not subscriptable

: 